# Przygotowanie danych do modelowania


## Wprowadzenie do przetwarzania wstępnego danych tabelarycznych

Przetwarzanie wstępne danych tabelarycznych jest kluczowym etapem w procesie budowania modeli uczenia maszynowego. Polega na transformacji surowych danych do formatu, który jest bardziej odpowiedni i zrozumiały dla algorytmów uczenia maszynowego. Dane rzeczywiste często zawierają braki, szumy i niespójności, które mogą negatywnie wpływać na wydajność modelu. Dlatego też, odpowiednie przygotowanie danych przed treningiem jest niezbędne do uzyskania optymalnych wyników.

### Setup: importy i dane

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

import kagglehub

Dane do pobrania z https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

In [ ]:
from google.colab import drive

# Podłącz Google Drive
drive.mount('/content/drive')

# Wstaw opowiednią ścieżkę do pliku na swoich GDrivie
housing_data = pd.read_csv('/content/drive/My Drive/Dydaktyka/OAI-III/housing/train.csv')
print(housing_data.shape)
print(housing_data.columns)
display(housing_data.head())

htarget = 'SalePrice'

### Podział danych na zbiór treningowy i testowy

Podział danych na zbiór treningowy i testowy jest kluczowym krokiem w procesie oceny modelu uczenia maszynowego. Zbiór treningowy jest wykorzystywany do nauki parametrów modelu, podczas gdy zbiór testowy służy do niezależnej oceny wydajności modelu na danych, których model wcześniej nie widział. Taki podział pozwala uniknąć przetrenowania (overfittingu), czyli sytuacji, w której model zbyt dobrze dopasowuje się do danych treningowych, tracąc zdolność generalizacji na nowe dane. Typowo dane są dzielone w proporcjach takich jak 80/20 lub 70/30 na zbiór treningowy i testowy.

In [ ]:
!pip install umap-learn

In [ ]:
from sklearn.model_selection import train_test_split

# Użyj przetworzonego zbioru danych housing_data
# W tym przykładzie użyjemy danych po imputacji braków dla kolumn numerycznych
# oraz kolumny docelowej 'SalePrice'
# Pamiętaj, że w pełnym potoku przetwarzania wstępnego należałoby użyć danych po wszystkich transformacjach (kodowaniu, skalowaniu)

# Zidentyfikuj cechy (wszystkie kolumny oprócz 'SalePrice' i 'Id') i zmienną docelową ('SalePrice')
# Użyjmy zbioru danych housing_data po imputacji braków numerycznych
X = housing_data.drop(['SalePrice', 'Id'], axis=1)  # Cechy
y = housing_data['SalePrice']  # Zmienna docelowa

# Podziel dane na zbiór treningowy i testowy
# test_size=0.2 oznacza, że 20% danych trafi do zbioru testowego, a 80% do treningowego
# random_state=42 zapewnia powtarzalność podziału
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Kształt zbioru treningowego cech (X_train):", X_train.shape)
print("Kształt zbioru testowego cech (X_test):", X_test.shape)
print("Kształt zbioru treningowego zmiennej docelowej (y_train):", y_train.shape)
print("Kształt zbioru testowego zmiennej docelowej (y_test):", y_test.shape)

### Obsługa brakujących wartości

Brakujące wartości są powszechnym problemem w rzeczywistych zbiorach danych. Mogą one wystąpić z różnych powodów, takich jak błędy podczas zbierania danych, niedostępność informacji lub błędy wprowadzania danych. Obecność brakujących wartości może prowadzić do błędnych wyników analizy lub obniżyć wydajność modeli uczenia maszynowego. Dlatego ważne jest, aby odpowiednio je obsłużyć przed dalszym przetwarzaniem danych. Typowe metody obsługi brakujących wartości obejmują ich usunięcie (całych wierszy lub kolumn z brakami) lub imputację (zastąpienie brakujących wartości wartościami szacunkowymi, takimi jak średnia, mediana lub wartość najczęściej występująca).

In [ ]:
# Sprawdź brakujące wartości w zbiorze danych housing_data
print("Missing values percentage per column in housing_data:")
nulls = X_train.isnull().mean()
display((nulls[nulls > 0] * 100).round(2))

# Możesz dalej przetwarzać brakujące wartości tutaj, na przykład:

# Usuń kolumny z wysokim procentem brakujących wartości:
col_above_threshold = (nulls[nulls > 50]).index # np. usuń kolumny z ponad 50% brakujących wartości
X_train_cleaned = X_train.drop(col_above_threshold, axis=1)
X_test_cleaned = X_test.drop(col_above_threshold, axis=1)

# Imputacja brakujących wartości średnią dla kolumn numerycznych:
numerical_cols_with_na = X_train_cleaned.select_dtypes(include=np.number).columns[X_train_cleaned.select_dtypes(include=np.number).isnull().any()]
for col in numerical_cols_with_na:
    X_train_cleaned[col] = X_train_cleaned[col].fillna(X_train_cleaned[col].dropna().mean())
    X_test_cleaned[col] = X_test_cleaned[col].fillna(X_train_cleaned[col].dropna().mean())

# Imputacja brakujących modą dla kolumn czynnikowych:
cathegorical_cols_with_na = X_train_cleaned.select_dtypes(include=np.object_).columns[X_train_cleaned.select_dtypes(include=np.object_).isnull().any()]
for col in cathegorical_cols_with_na:
    X_train_cleaned[col] = X_train_cleaned[col].fillna(X_train_cleaned[col].dropna().mode().values[0])
    X_test_cleaned[col] = X_test_cleaned[col].fillna(X_train_cleaned[col].dropna().mode().values[0])


print("Missing values percentage per column in housing_data:")
nulls = X_train_cleaned.isnull().mean()
display((nulls[nulls > 0] * 100).round(2))

### Kodowanie cech kategorialnych

Cechy kategorialne reprezentują dane, które można podzielić na grupy lub kategorie. Algorytmy uczenia maszynowego zazwyczaj wymagają numerycznych danych wejściowych, dlatego konieczne jest przekształcenie cech kategorialnych na format liczbowy. Istnieje wiele metod kodowania cech kategorialnych, a wybór odpowiedniej metody zależy od charakteru danych i algorytmu uczenia maszynowego. Dwie powszechnie stosowane metody to:

*   **One-Hot Encoding:** Tworzy nowe kolumny binarne dla każdej unikalnej kategorii w oryginalnej kolumnie. Jest to przydatne, gdy nie ma naturalnego porządku między kategoriami.
*   **Label Encoding:** Przypisuje unikalną liczbę całkowitą każdej kategorii. Jest to odpowiednie, gdy istnieje naturalny porządek między kategoriami (np. mały, średni, duży).

In [ ]:
# Zilustruj kodowanie zmiennych kategorialnych na podstawie housing_data

# Zidentyfikuj kolumny kategorialne
# Użyj type object, ponieważ kolumny kategorialne często są w tym formacie w pandas
categorical_cols = X_train_cleaned.select_dtypes(include=np.object_).columns

print("Kolumny kategorialne w zbiorze danych housing_data:")
print(categorical_cols)

for col in categorical_cols:
  vals = X_train_cleaned[col].unique()
  X_train_cleaned[col] = pd.Categorical(X_train_cleaned[col], categories=vals)
  X_test_cleaned[col] = pd.Categorical(X_test_cleaned[col], categories=vals)

# Zastosuj One-Hot Encoding do kolumn kategorialnych bez naturalnego porządku (nominalnych)
# One-Hot Encoding tworzy nowe kolumny binarne dla każdej unikalnej kategorii.
# Jest to odpowiednie, gdy nie ma naturalnego porządku między kategoriami.
# Wybierzmy przykładowe kolumny nominalne do demonstracji

X_train_cleaned = pd.get_dummies(X_train_cleaned, columns=categorical_cols, drop_first=True) # drop_first=True zapobiega pułapce zmiennej pozornej
X_test_cleaned = pd.get_dummies(X_test_cleaned, columns=categorical_cols, drop_first=True) # drop_first=True zapobiega pułapce zmiennej pozornej

print("\nDataFrame po One-Hot Encoding (przykładowe kolumny):")
display(X_test_cleaned.head())

# Uwaga: W pełnym potoku przetwarzania wstępnego dla zbioru housing_data,
# należałoby dokładnie przeanalizować wszystkie kolumny kategorialne
# i zastosować odpowiednie metody kodowania (One-Hot, Label z mapowaniem ręcznym, itp.)
# oraz obsłużyć brakujące wartości w bardziej zaawansowany sposób, jeśli to konieczne.

### Skalowanie cech numerycznych

Skalowanie cech numerycznych jest ważnym etapem przetwarzania wstępnego, ponieważ wiele algorytmów uczenia maszynowego jest wrażliwych na skalę danych wejściowych. Cechy o większych wartościach mogą dominować nad cechami o mniejszych wartościach, co może prowadzić do błędnych lub mniej dokładnych wyników. Skalowanie przekształca wartości cech do określonego zakresu lub rozkładu, zapewniając, że wszystkie cechy mają podobny wpływ na proces uczenia. Dwie popularne metody skalowania to:
- standaryzacja (StandardScaler), która skaluje dane tak, aby miały średnią 0 i odchylenie standardowe 1
$$\frac {X - X.mean()} {X.std()}$$
- oraz normalizacja (MinMaxScaler), która skaluje dane do określonego zakresu, zazwyczaj od 0 do 1.
$$\frac {X - X.min()} {X.max() - X.min()}$$

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Zidentyfikuj kolumny numeryczne w zbiorze danych housing_data (z wyłączeniem kolumny 'Id')
numerical_cols = X_train_cleaned.select_dtypes(include=np.number).columns.tolist()
if 'Id' in numerical_cols:
    numerical_cols.remove('Id')

# Wybierz tylko kolumny numeryczne do skalowania
X_train_cleaned_ = X_train_cleaned[numerical_cols].copy()
X_test_cleaned_ = X_test_cleaned[numerical_cols].copy()

# Zilustruj standaryzację przy użyciu StandardScaler
# Standaryzacja skaluje dane tak, aby miały średnią 0 i odchylenie standardowe 1.
# Jest szczególnie użyteczna dla algorytmów, które zakładają, że dane mają rozkład normalny
# lub są wrażliwe na wariancję cech, takich jak SVM, regresja logistyczna, czy algorytmy oparte na odległościach.
scaler_standard = StandardScaler()
X_train_cleaned_standard = pd.DataFrame(scaler_standard.fit_transform(X_train_cleaned_), columns=numerical_cols)
X_test_cleaned_standard = pd.DataFrame(scaler_standard.transform(X_test_cleaned_), columns=numerical_cols)

print("\nDataFrame po standaryzacji (StandardScaler):")
display(X_train_cleaned_standard.head())

scaler_minmax = MinMaxScaler()
X_train_cleaned_minmax = pd.DataFrame(scaler_minmax.fit_transform(X_train_cleaned_), columns=numerical_cols)
X_test_cleaned_minmax = pd.DataFrame(scaler_minmax.transform(X_test_cleaned_), columns=numerical_cols)

print("\nDataFrame po standaryzacji (StandardScaler):")
display(X_train_cleaned_minmax.head())

## Redukcja wymiarowości

Redukcja wymiarowości jest techniką stosowaną w celu zmniejszenia liczby zmiennych (cech) w zbiorze danych. Jest to przydatne, gdy zbiór danych ma bardzo wiele cech, co może prowadzić do problemu "przekleństwa wymiarowości". Redukcja wymiarowości pomaga zmniejszyć złożoność obliczeniową, zredukować szum w danych i ułatwić wizualizację. Popularne metody redukcji wymiarowości to Analiza Głównych Składowych (PCA) i t-SNE.

### Analiza Głównych Składowych (PCA)

PCA (Principal Component Analysis) to popularna technika redukcji wymiarowości, która ma na celu przekształcenie zbioru potencjalnie skorelowanych zmiennych w zbiór nieskorelowanych zmiennych, zwanych głównymi składowymi.

Główne składowe są liniowymi kombinacjami oryginalnych cech i są tak konstruowane, aby pierwsza składowa miała największą możliwą wariancję (zawierała najwięcej informacji o zmienności danych), druga składowa miała drugą co do wielkości wariancję i była nieskorelowana z pierwszą, i tak dalej.

Proces PCA obejmuje zazwyczaj następujące kroki:
1.  **Standaryzacja danych:** Skalowanie danych tak, aby każda cecha miała średnią zero i wariancję jeden. Jest to ważne, ponieważ PCA jest wrażliwe na skalę cech.
2.  **Obliczenie macierzy kowariancji (lub korelacji):** Macierz ta pokazuje, jak poszczególne cechy są ze sobą powiązane.
3.  **Obliczenie wektorów i wartości własnych:** Wektory własne macierzy kowariancji reprezentują kierunki (główne składowe) w przestrzeni danych, a wartości własne odpowiadają wielkości wariancji w tych kierunkach.
4.  **Sortowanie wektorów własnych:** Sortowanie wektorów własnych malejąco według odpowiadających im wartości własnych. Najważniejsze składowe to te z największymi wartościami własnymi.
5.  **Wybór podzbioru wektorów własnych:** Wybór K wektorów własnych o największych wartościach własnych, gdzie K to docelowa liczba wymiarów.
6.  **Transformacja danych:** Przekształcenie oryginalnych danych na nową przestrzeń zdefiniowaną przez wybrane wektory własne.

PCA jest użyteczne do redukcji szumu, kompresji danych, wizualizacji danych wielowymiarowych oraz jako krok wstępny przed zastosowaniem innych algorytmów uczenia maszynowego.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Użyj przetworzonych danych numerycznych z poprzednich kroków (po skalowaniu)
# Zmienna housing_data_scaled_standard zawiera numeryczne kolumny housing_data po standaryzacji
df_features = X_train_cleaned_standard.copy()

# Zastosuj PCA
# Zredukuj do 2 komponentów dla wizualizacji
pca = PCA(n_components=2)
principal_components = pca.fit_transform(df_features)
df_pca = pd.DataFrame(data = principal_components, columns = ['principal component 1', 'principal component 2'])

print("\nDataFrame po PCA (2 główne składowe):")
display(df_pca.head())

# Opcjonalnie: Wizualizacja wyników PCA (jeśli masz więcej punktów danych i chcesz zwizualizować klastry lub rozkład)
plt.figure(figsize=(8, 6))
plt.scatter(df_pca['principal component 1'], df_pca['principal component 2'])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA zbioru danych housing_data (kolumny numeryczne)')
plt.show()

#### Interpretacja komponentów głównych

Tutaj można dodać interpretację znaczenia poszczególnych komponentów głównych w kontekście analizowanych danych. Na przykład, które oryginalne cechy mają największy wpływ na pierwszy komponent główny? Co reprezentują te komponenty?

In [ ]:
for component in pca.components_:
  plt.bar(df_features.columns, component)
  plt.xticks(rotation=90)
  plt.title('New variable in PCA')
  plt.ylabel('weight')
  plt.show()

### t-Distributed Stochastic Neighbor Embedding (t-SNE)

Wyobraź sobie, że masz zbiór danych z wieloma cechami, a chcesz zobaczyć, jak te punkty danych są ze sobą powiązane w przestrzeni o mniejszej liczbie wymiarów (np. na płaszczyźnie 2D). PCA pomaga znaleźć najważniejsze kierunki zmienności, ale czasami nie najlepiej zachowuje lokalne relacje między punktami (czyli to, które punkty są "sąsiadami").

t-SNE działa trochę inaczej. Skupia się na tym, żeby punkty, które są blisko siebie w oryginalnej przestrzeni o wielu wymiarach, pozostały blisko siebie na wykresie 2D (lub 3D). Jednocześnie stara się, żeby punkty, które są daleko od siebie w oryginalnej przestrzeni, pozostały daleko na wykresie.

Można to sobie wyobrazić tak, jakbyś próbował rozłożyć trójwymiarową kulę na płaską kartkę papieru. Nie da się tego zrobić idealnie bez zniekształceń. t-SNE próbuje zminimalizować te zniekształcenia, koncentrując się na zachowaniu "sąsiedztwa".

Jak to robi?
1. Najpierw oblicza, jak bardzo "podobne" są do siebie wszystkie pary punktów w oryginalnej, wielowymiarowej przestrzeni. Punkty blisko siebie są bardzo podobne, punkty daleko - mało podobne.
2. Następnie tworzy mapę tych punktów w przestrzeni o mniejszej liczbie wymiarów (np. 2D) i też oblicza ich "podobieństwo".
3. Na koniec, t-SNE przesuwa punkty na mapie 2D tak, aby "podobieństwo" punktów na mapie było jak najbardziej zbliżone do "podobieństwa" punktów w oryginalnej przestrzeni. Robi to iteracyjnie, czyli krok po kroku poprawia rozmieszczenie punktów.

W efekcie dostajemy wykres, na którym skupiska punktów reprezentują grupy podobnych obiektów. t-SNE jest świetne do wizualizacji złożonych danych i odkrywania ukrytych struktur lub klastrów, ale trudniej jest interpretować same osie wykresu tak jak w PCA.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Użyj przetworzonych danych numerycznych z poprzednich kroków (po skalowaniu)
# Zmienna housing_data_scaled_standard zawiera numeryczne kolumny housing_data po standaryzacji
df_features = X_train_cleaned_standard.copy()

# Zastosuj t-SNE
# Zredukuj do 2 komponentów dla wizualizacji
# Wartości perplexity i max_iter mogą wymagać dostrojenia w zależności od danych
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=300)
principal_components_tsne = tsne.fit_transform(df_features)
df_tsne = pd.DataFrame(data = principal_components_tsne, columns = ['tsne component 1', 'tsne component 2'])

print("\nDataFrame po t-SNE (2 komponenty):")
display(df_tsne.head())

# Wizualizacja wyników t-SNE
plt.figure(figsize=(8, 6))
plt.scatter(df_tsne['tsne component 1'], df_tsne['tsne component 2'])
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE zbioru danych housing_data (kolumny numeryczne)')
plt.show()

### UMAP (Uniform Manifold Approximation and Projection) - intuicyjnie

UMAP to kolejna technika redukcji wymiarowości, podobna do t-SNE, ale często szybsza i lepiej zachowująca globalną strukturę danych.

Wyobraź sobie, że Twoje dane o wielu wymiarach leżą na skomplikowanej, pofałdowanej powierzchni w tej wielowymiarowej przestrzeni. UMAP próbuje znaleźć sposób, żeby "rozłożyć" tę pofałdowaną powierzchnię na płasko (np. na 2D), zachowując jednocześnie odległości między punktami.

Jak to robi?
1. **Buduje wykres "sąsiadów":** Najpierw UMAP tworzy wykres, gdzie każdy punkt danych jest połączony z jego najbliższymi sąsiadami w oryginalnej, wielowymiarowej przestrzeni. Siła połączenia zależy od tego, jak blisko są te punkty.
2. **Tworzy wykres 2D:** Następnie UMAP próbuje zbudować podobny wykres w przestrzeni o mniejszej liczbie wymiarów (np. 2D).
3. **Dopasowuje wykresy:** UMAP dopasowuje rozmieszczenie punktów na wykresie 2D tak, aby struktura "sąsiadów" na wykresie 2D była jak najbardziej podobna do struktury "sąsiadów" w oryginalnej, wielowymiarowej przestrzeni.

Podobnie jak t-SNE, UMAP jest świetne do wizualizacji i odkrywania grup w danych. Często daje lepsze wyniki w zachowaniu zarówno lokalnej, jak i globalnej struktury danych w porównaniu do t-SNE.

W skrócie, UMAP próbuje stworzyć "mapę" Twoich wielowymiarowych danych w niższej wymiarowości, koncentrując się na zachowaniu relacji między punktami (kto jest czyim sąsiadem i jak blisko).

In [ ]:
import umap
import matplotlib.pyplot as plt

# Użyj przetworzonych danych numerycznych z poprzednich kroków (po skalowaniu)
# Zmienna housing_data_scaled_standard zawiera numeryczne kolumny housing_data po standaryzacji
df_features = X_train_cleaned_standard.copy()

# Zastosuj UMAP
# Zredukuj do 2 komponentów dla wizualizacji
# Parametry n_neighbors i min_dist mogą wymagać dostrojenia
reducer = umap.UMAP(n_components=2, random_state=42)
embedding = reducer.fit_transform(df_features)

df_umap = pd.DataFrame(embedding, columns=['umap component 1', 'umap component 2'])

print("\nDataFrame po UMAP (2 komponenty):")
display(df_umap.head())

# Wizualizacja wyników UMAP
plt.figure(figsize=(8, 6))
plt.scatter(df_umap['umap component 1'], df_umap['umap component 2'])
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.title('UMAP zbioru danych housing_data (kolumny numeryczne)')
plt.show()

## Ćwiczenia

Po zapoznaniu się z podstawowymi technikami przetwarzania wstępnego danych tabelarycznych, proponujemy następujące ćwiczenia, które pozwolą utrwalić zdobytą wiedzę i umiejętności:

**Zadanie 1: Analiza i Przetwarzanie Danych Titanic**

Wykorzystując zbiór danych Titanic (https://www.kaggle.com/datasets/yasserh/titanic-dataset), przeprowadź kompleksowe przetwarzanie wstępne danych, uwzględniając poznane techniki:

1.  **Ładowanie i wstępna analiza**: Załaduj dane do DataFrame i przeprowadź wstępną analizę:
    *   Sprawdź rozmiar zbioru danych (liczba wierszy i kolumn).
    *   Wyświetl pierwsze kilka wierszy, aby zapoznać się ze strukturą danych.
    *   Sprawdź typy danych poszczególnych kolumn.
    *   Zidentyfikuj kolumny numeryczne i kategoryczne.
2.  **Obsługa brakujących wartości**:
    *   Zidentyfikuj kolumny z brakującymi wartościami i wyświetl procent braków dla każdej z nich.
    *   Zastosuj odpowiednie strategie imputacji dla kolumn numerycznych (np. średnia, mediana) i kategorialnych (np. moda, stała wartość, np. "Missing"). Uzasadnij swój wybór dla każdej kolumny.
3.  **Kodowanie cech kategorialnych**:
    *   Zastosuj One-Hot Encoding do kolumn kategorialnych, które nie mają naturalnego porządku.
    *   Zastosuj Label Encoding lub inne odpowiednie kodowanie (np. mapowanie ręczne) do kolumn kategorialnych, które mają naturalny porządek (jeśli takie występują w zbiorze danych Titanic).
4.  **Skalowanie cech numerycznych**:
    *   Zastosuj standaryzację (StandardScaler) do wszystkich kolumn numerycznych.
    *   Zastosuj normalizację (MinMaxScaler) do tych samych kolumn numerycznych w osobnym etapie. Porównaj rozkłady danych po obu transformacjach (np. używając histogramów).
5.  **Podział danych**: Podziel przetworzone dane na zbiór treningowy i testowy w proporcji 80/20 lub 70/30.

**Zadanie 2: Redukcja Wymiarowości i Wizualizacja**

Kontynuuj pracę na przetworzonym zbiorze danych Titanic (po zastosowaniu wszystkich kroków przetwarzania wstępnego):

1.  **Zastosowanie PCA**: Zastosuj PCA do przetworzonych danych, redukując wymiarowość do 2 lub 3 komponentów. Zwizualizuj wyniki PCA na wykresie punktowym. Spróbuj zinterpretować znaczenie pierwszych komponentów głównych, analizując wagi oryginalnych cech.
2.  **Zastosowanie t-SNE**: Zastosuj t-SNE do przetworzonych danych, redukując wymiarowość do 2 komponentów. Zwizualizuj wyniki t-SNE na wykresie punktowym. Czy widzisz wyraźne skupiska punktów odpowiadające np. klasie przeżycia?
3.  **Zastosowanie UMAP**: Zastosuj UMAP do przetworzonych danych, redukując wymiarowość do 2 komponentów. Zwizualizuj wyniki UMAP na wykresie punktowym. Porównaj wizualizacje z t-SNE i PCA - która metoda lepiej oddaje strukturę danych w niskowymiarowej przestrzeni?
4.  **Dyskusja**: Napisz krótkie podsumowanie porównujące zastosowane metody redukcji wymiarowości w kontekście zbioru danych Titanic. Kiedy warto zastosować każdą z nich? Jakie są ich zalety i wady w tym konkretnym przypadku?

Powodzenia!